<a href="https://colab.research.google.com/github/87tana/YOLO11n-Bone-Fracture-Detection-Model/blob/main/basemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation , Import Libraries and Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/my_drive', force_remount=True)

# Navigate to the project directory
%cd '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/'

# Define dataset path
dataset_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset'




Mounted at /content/my_drive
/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection


In [2]:
!pip install ultralytics torch torchvision torchaudio mlflow pyngrok pyyaml --quiet



In [3]:
# General libraries
import os
import time
import numpy as np
#import mlflow
from PIL import Image

from ultralytics import YOLO
import ultralytics

import datetime

import subprocess

# MLflow
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow.pyfunc


# Google Colab tools
from google.colab import drive

# For ngrok to expose the MLflow UI remotely
#from pyngrok import ngrok

# Visualization
import matplotlib.pyplot as plt


# Train YOLOv11n

- This part starts the experiment, logs parameters, and trains the YOLOv11n model.

In [4]:
def train_yolo(model_path, dataset_path, log_path, exp_id=datetime.datetime.now().strftime('%Y%m%d%H%M%S'), epochs=50, lr=0.001, optimizer='AdamW', resume=False):

    with mlflow.start_run(run_name=exp_id):
        # Log hyperparameters
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("optimizer", optimizer)
        mlflow.log_param("img_size", 640)
        mlflow.log_param("batch_size", 64)

        # Start timer for training duration
        start_time = time.time()

        if resume:
            # Load a model
            model = YOLO(model_path)  # load a partially trained model
            # Resume training
            results = model.train(resume=True)
        else:
            # Hyperparameters for training YOLOv11n
            params = {
                'exp_id': exp_id,
                'epochs': epochs,
                'img_size': 640,
                'batch_size': 64,
                'lr': lr,
                'optimizer': optimizer,
                'save_period': 10
            }

            # Load the YOLOv11n model
            model = YOLO(model_path)  # Load YOLOv11n model with custom weights

            # Train the model with updated parameters
            results = model.train(
                data=os.path.join(dataset_path, 'data.yaml'),
                epochs=params['epochs'],
                imgsz=params['img_size'],
                batch=params['batch_size'],
                lr0=params['lr'],
                optimizer=params['optimizer'],
                project=log_path,
                name=params['exp_id'],
                save_period=params['save_period'],
            )

        # Log training time
        training_duration = time.time() - start_time
        mlflow.log_metric("training_duration", training_duration)
        print(f"Training completed in {training_duration:.2f} seconds")

        return results


In [5]:
# Set MLflow tracking
mlflow_tracking_dir = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/runs/mlflow'
mlflow.set_tracking_uri(f"file://{mlflow_tracking_dir}")


mlflow.set_experiment("Base_Bone_Fraction_Detection")


2024/12/25 20:52:07 INFO mlflow.tracking.fluent: Experiment with name 'Base_Bone_Fraction_Detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/runs/mlflow/669269712119322704', creation_time=1735159927933, experiment_id='669269712119322704', last_update_time=1735159927933, lifecycle_stage='active', name='Base_Bone_Fraction_Detection', tags={}>

In [6]:
mlflow.end_run()

# Dataset path (ensure this is where your data is located)
dataset_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset'

# Pre-trained model path
model_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/models/yolo11n.pt'

# Log path
log_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints'

# Parameters
exp_id='02'
epochs= 60
lr=0.001


results = train_yolo(model_path,
                     dataset_path,
                     log_path,
                     exp_id=exp_id,
                     epochs=epochs,
                     lr=lr
                     )


Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/models/yolo11n.pt, data=/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/data.yaml, epochs=60, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints, name=02, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream

train: Scanning /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/train/labels.cache... 3779 images, 126 backgrounds, 0 corrupt: 100%|██████████| 3779/3779 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/valid/labels.cache... 835 images, 5 backgrounds, 0 corrupt: 100%|██████████| 835/835 [00:00<?, ?it/s]


Plotting labels to /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/02/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)


...

      60/60      8.81G      0.516     0.3305      1.123          3        640: 100%|██████████| 60/60 [01:18<00:00,  1.31s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:13<00:00,  1.89s/it]

                   all        835       1373      0.576      0.701      0.512       0.27



60 epochs completed in 1.635 hours.
Optimizer stripped from /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/02/weights/last.pt, 5.5MB
Optimizer stripped from /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/02/weights/best.pt, 5.5MB

Validating /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/02/weights/best.pt...
Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:16<00:00,  2.41s/it]


                   all        835       1373      0.593      0.693      0.544       0.29
           No_fracture        187        202      0.421      0.886      0.495      0.362
              Fracture        643       1171      0.766        0.5      0.593      0.217
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 6.1ms postprocess per image
Results saved to /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/02
MLflow: results logged to file:///content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'
Training completed in 5985.36 seconds


In [13]:
# Run MLflow UI in the background
import subprocess
from pyngrok import ngrok

subprocess.Popen(["mlflow", "ui", "--backend-store-uri", mlflow_tracking_dir])

# Set up Ngrok for remote access
NGROK_AUTH_TOKEN = "2qGXY4i9cF4CQHjBRFLtxjuQCTz_Z9PruXmBce21Hfpju7AY"  # Replace with your Ngrok token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for accessing MLflow UI
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)

print("MLflow Tracking UI:", ngrok_tunnel.public_url)

# Terminate open tunnels if exist
ngrok.kill()


MLflow Tracking UI: https://2282-34-168-102-55.ngrok-free.app


# inference

In [ ]:
# nference on a specific image from your training dataset using a trained YOLO model

image_list = os.listdir('/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/train/images')

# Run inference on a test image
test_image_path = os.path.join("/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/train/images/", image_list[5])  # Example image index

# Load the YOLOv11n model (ensure the weights file exists at the given path)
model_inf_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/YOLOv11n_experiment/weights/last.pt'
model_inf = YOLO(model_inf_path)  # Load YOLOv11n model with custom weights
results = model_inf(test_image_path)  # Get predictions for the test image

# Visualize the results
for r in results:
    # Plot the prediction results
    im_bgr = r.plot()  # Convert to BGR
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # Convert to RGB
    r.show()  # Show the result


In [27]:
"""
def train_yolo(model_path, dataset_path, log_path, exp_id=datetime.datetime.now().strftime('%Y%m%d%H%M%S'), epochs=50, lr=0.001, optimizer='AdamW', resume=False):


  # Start timer for training duration
  start_time = time.time()

  if resume:

    # Load a model
    model = YOLO(model_path)# load a partially trained model
    # Resume training
    results = model.train(resume=True)

  else:

    # Hyperparameters for training YOLOv11n
    params = {
        'exp_id': exp_id,
        'epochs': epochs,
        'img_size': 640,
        'batch_size': 64,
        'lr': lr,
        'optimizer': optimizer,
        'save_period': 1
    }


    # Start timer for training duration
    start_time = time.time()

    # Load the YOLOv11n model (ensure the weights file exists at the given path)
    model = YOLO(model_path)  # Load YOLOv11n model with custom weights

    model.callbacks['on_train_epoch_end']

    # Train the model with updated parameters
    results = model.train(
        data = os.path.join(dataset_path, 'data.yaml'),
        epochs =params['epochs'],
        imgsz=params['img_size'],
        batch =params['batch_size'],
        lr0= params['lr'],
        optimizer=params['optimizer'],
        project= log_path,
        name=params['exp_id'],
        save_period=params['save_period'],
    )

  # Log training time
  training_duration = time.time() - start_time
  print(f"Training completed in {training_duration:.2f} seconds")

  return results
"""


            # Define a custom callback to log metrics for each epoch
            #def mlflow_callback(trainer):
            #    metrics = trainer.metrics
            #    current_epoch = trainer.epoch
            #    print('metrics:',metrics)
            #    mlflow.log_metric("train_loss", metrics['val/box_loss'], step=current_epoch)
                # mlflow.log_metric("train_cls_loss", metrics.cls_loss,
                # step=current_epoch) mlflow.log_metric("train_focal_loss",
                # metrics.focal_loss, step=current_epoch)
                # mlflow.log_metric("train_total_loss", metrics.loss,
                # step=current_epoch) mlflow.log_metric("val_box_loss",
                # metrics.val_box_loss, step=current_epoch)
                # mlflow.log_metric("val_cls_loss", metrics.val_cls_loss,
                # step=current_epoch) mlflow.log_metric("val_focal_loss",
                # metrics.val_focal_loss, step=current_epoch)
                # mlflow.log_metric("val_total_loss", metrics.val_loss,
                # step=current_epoch)
                #mlflow.log_metric("lr", trainer.optimizer.param_groups[0]['lr'], step=current_epoch)

            # Attach the custom callback
            #model.add_callback('on_train_epoch_end', mlflow_callback)

